<a href="https://colab.research.google.com/github/aidot-kr/MLStudy/blob/master/BERT_%ED%86%A0%EB%8B%A5%ED%86%A0%EB%8B%A5%EB%B4%87_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>[BERT를 활용한 텍스트 분류]</h1>
<h2>🥰토닥토닥봇 만들기 - 데이터 전처리 </h2>

## #1. 실습 준비
이번 모듈에서는 다운받은 엑셀 파일을 전처리해서 BERT 학습이 가능한 인풋으로 만드는 과정을 진행할 것입니다.

전처리된 데이터셋을 저장할 수 있도록 구글 드라이브를 마운트하고, 필요한 라이브러리를 설치해보겠습니다.

In [ ]:
## 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## Tensorflow에서 BERT를 사용할 수 있는 라이브러리인 bert-for-tf2와 konlpy 패키지 설치
## 실습에 필요한 라이브러리 로딩
!pip install bert-for-tf2
!pip install konlpy


import json
import numpy as np
import tensorflow as tf

import bert
import tensorflow_hub as hub

     |████████████████████████████████| 40kB 1.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=6a4b49f235a9cf47bb4665387a0cca12845fd2332b071519a3370ec84300c41b
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=afef27f7b49499ba76f072347171bbb6b35ff582f81d152213eb3c6beb5b4c21
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=d155ffdc007b6dab4485766acde69ab2ac88494aeb13363175287a5965ecf504
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 19.4MB 56.0MB/s 
     |███████████████████████████

<font color = "red"><b>
gdrive/NLP에</font>    

- "Wellness_data_train.json"
- "Wellness_data_test.json"
- "Wellness_response.json"   

데이터가 없는 경우 아래의 코드 복사해 실행</b>

```
"""MAKE DATASET"""
import random
import pandas as pd
random.seed(1)

def isNaN(num):
    return num != num

EXCEL_FILE_NALE = "/content/웰니스_대화_스크립트_데이터셋.xlsx"
data = pd.read_excel(EXCEL_FILE_NALE)

DATA = []
RESPONSE = {}

for i in range(len(data["구분"])):
  label = data["구분"][i]
  label_split = label.split("/")
  label_1 = "/".join(label_split[:2])
  sent = data["유저"][i]
  if label_1 != "모호함":
    DATA.append(["Sent_{}".format(i), sent, label_1, label])
    if label_1 in RESPONSE:  
      if not isNaN(data["챗봇"][i]): 
        RESPONSE[label_1].append(data["챗봇"][i])
    else: 
      if not isNaN(data["챗봇"][i]):  
        RESPONSE[label_1] = [data["챗봇"][i]]

"""random shuffle & make them into train/test set"""
labels = [dat[2] for dat in DATA]
from sklearn.model_selection import train_test_split
train, test = train_test_split(DATA, random_state = 2020, stratify = labels, test_size = 400)

with open("/content/gdrive/My Drive/NLP/Wellness_data_train.json","w") as f:
  f.write(json.dumps(train))
with open("/content/gdrive/My Drive/NLP/Wellness_data_test.json","w") as f:
  f.write(json.dumps(test))
with open("/content/gdrive/My Drive/NLP/Wellness_response.json","w") as f:
  f.write(json.dumps(RESPONSE))
```

## #2. 사전학습된 BERT 모델 로드하기
저희는 구글에서 공개한 다국어 모델인 BERT-multilingual-cased 모델을 활용해 의도분류 모델로 fine-tuning할 것입니다.

Tensorflow HUB에서는 사전학습된 모델 과 토크나이저 등 학습에 필요한 것들를 제공하고 있습니다.

아래 코드는 <b>hub.KerasLayer</b> 함수를 통해 다국어 모델을 다운로드하는 코드입니다.

이때 fine-tuning이 가능하도록 레이어의 trainable 옵션을 <font color="blue">True</font>로 설정합니다.

In [ ]:
BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2'

# BERT layer 가져오기
bert_layer = hub.KerasLayer(BERT_MODEL_HUB, trainable=True)

## #3. BERT 토크나이저 로드하기
BERT 모델을 fine-tuning하기 위해서는 사전학습된 BERT가 사용한 단어사전을 이용해 인풋을 처리해야 하겠지요?

BERT 토크나이저를 로드하고, 한국어 형태소분석과 WordPiece Tokenizer을 적용해 토크나이즈하는 연습을 해보겠습니다.

#### Step 1. 토크나이저 로딩하기
- hub를 통해 로드한 bert_layer에는 사전학습에 활용한 단어사전 정보가 담겨있습니다.
- bert-for-tf2 라이브러리에서 제공하는 bert_tokenization 함수를 이용해 BERT 토크나이저를 로드하겠습니다.
- 아래 코드는 tokenizer라는 이름으로 BERT 토크나이저를 로딩하고, 단어사전 내의 토큰들을 확인하는 코드입니다.

In [ ]:
from  bert.tokenization import bert_tokenization

# vocab_file 가져오기
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# 소문자화를 하는지 여부 가져오기
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

print("vocab file:", vocab_file) # vocab 사전을 저장한 경로
print("do_lower_case:", do_lower_case) # 소문자화 여부 (다국어 모델의 경우 False임)

# 토크나이저 로딩
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

# vocab 사전 확인하기
print("단어사전에 있는 토큰 개수:", len(tokenizer.vocab))
print("예시:", list(tokenizer.vocab.keys())[:300])

vocab file: b'/tmp/tfhub_modules/3e9209b9f2a53dfa4e6d93250dfceb5e64d73b66/assets/vocab.txt'
do_lower_case: False
단어사전에 있는 토큰 개수: 119547
예시: ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unu

#### Step 2. 형태소 분석 + BERT 토크나이즈 실행하기

- 한국어의 경우, 형태소 분석을 진행한 문장에 BERT 토크나이즈를 적용하는 것이 좋은 것으로 알려져 있습니다.
- konlpy의 Okt 형태소 분석기를 통해 형태소 분석을 진행하고, BERT 토크나이저를 적용하는 코드를 실행해보세요.

In [ ]:
""" 형태소 분석 함수 """
from konlpy.tag import Okt
okt=Okt()
def tokenize(lines):
  return okt.morphs(lines)

sentence = "버트로 토크나이즈하는 예제"
print("** 원본 문장 :", sentence)

# basic_tokenizer로 문장 쪼개기
tokenized_sentence = tokenize(sentence)
print("** 형태소 분석 결과 :",tokenized_sentence)

# BPE로 문장 쪼개기
sub_tokens = tokenizer.tokenize(" ".join(tokenized_sentence))
print("** BERT 토크나이즈 결과 :", sub_tokens)

** 원본 문장 : 버트로 토크나이즈하는 예제
** 형태소 분석 결과 : ['버트', '로', '토크', '나', '이즈', '하', '는', '예제']
** BERT 토크나이즈 결과 : ['버', '##트', '로', '토', '##크', '나', '이', '##즈', '하', '는', '예', '##제']


#### Step 3. BERT 토크나이징된 토큰을 인풋 인덱스로 바꾸기

- 이제 토큰화된 토큰들을 모델이 처리할 수 있는 정수 인덱스로 매핑하겠습니다.
- 이 과정은 토크나이저의 convert_tokens_to_ids 매서드를 이용해 간단하게 진행할 수 있습니다.

In [ ]:
# 모델 인풋 인덱스로 바꾸기
input_ids = tokenizer.convert_tokens_to_ids(sub_tokens)
print("** 매핑된 인덱스:", input_ids)

** 매핑된 인덱스: [9336, 15184, 9202, 9873, 20308, 8982, 9638, 24891, 9952, 9043, 9576, 17730]


## #4. 텍스트 분류 모델을 위한 BERT 인풋 생성하기 

이제 BERT 모델 fine-tuning에 필요한 세 가지 tensor들을 만들어보겠습니다. 

- Input_ids : BERT 토크나이징을 적용한 토큰에 대한 정수 인덱스
- Segment_ids : 각 토큰이 [문장 1]에 해당하는지 [문장 2]에 해당하는지 나타내는 인덱스
- Input_mask : 실제로 토큰이 존재하는 부분은 1, [PAD]로 채운 부분은 0으로 표시한 인덱스 


#### step 1. 데이터 읽어오기

In [ ]:
import json
filename = "/content/gdrive/My Drive/NLP/Wellness_data_train.json"

with open(filename) as f:
  train = json.loads(f.read())
print("# of Train =", len(train))

# of Train = 4651


In [ ]:
## 데이터셋 확인
print(train[:5])

[['Sent_4393', '뭔가 하루종일 이렇게 들뜬 기분이다 보니까 잠도 잘 안 와.', '증상/불면', '증상/불면'], ['Sent_603', '아무한테나 화내고 그러지는 않아.', '감정/분노', '감정/분노'], ['Sent_4224', '잠자리에 누워도 맨날 뒤척이고... 잠을 제대로 잘 수 있을 리가 없지.', '증상/불면', '증상/불면'], ['Sent_3849', '5일 전에는 새벽에 일어나서 화장실을 가다가 순간적으로 정신을 잃었어.', '증상/기절', '증상/기절'], ['Sent_666', '그냥 감정이입이 심하게 되고 불안감도 잘 느끼는 것 같아요.', '감정/불안감', '감정/불안감']]


#### Step 2. BERT 인풋 만들기 - 함수 정의

- 학습을 위해 자연어 문장에 BPE를 적용하고, input_ids, segment_ids, input_masks를 생성하는 함수를 정의하겠습니다.
- 이 함수에서는 학습을 위해 Ground Truth 라벨까지 정보를 InputFeatures라는 클래스를 이용해 저장하겠습니다.

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

<font color = "red">[MISSION] convert_examples_to_features 함수 완성하기</font>   

convert_examples_to_features는 
- examples (데이터의 리스트) 
- max_seq_length (인풋 시퀀스의 최대 시퀀스 길이)
- tokenizer (로딩한 BERT토크나이저)
- label_map (인덱스 매핑 사전)   

를 인풋으로 받아 InputFeatures를 반환하는 함수입니다.   
아래 코드에서 [★CODE] 부분을 수정하여 함수가 정상적으로 작동하도록 수정해보세요.

<i>** 정답은 Wire 페이지를 통해 확인할 수 있습니다. </i>


In [ ]:
def convert_examples_to_features(examples, max_seq_length, tokenizer, label_map):
    # features : 리턴할 InputFeatures를 담을 리스트
    features = []

    for (ex_index, example) in enumerate(examples):
        """
        함수의 인풋으로 들어오는 리스트는 [문장번호 - 문장 - 라벨]의 순서로 이루어져 있습니다.
        문장을 text 변수에 저장하고, 라벨을 label 변수에 저장하였습니다.
        label이 어떤 정수 인덱스에 해당하는지는 label_map을 통해 정보를 받아 label_id 변수에 저장합니다.
        """
        text = example[1] # 문장
        label = example[2] # 라벨
        label_id = label_map[label]

        ## STEP 1. 형태소 분석 + BPE를 통해 토큰화하기
        otk_tokenized_text = [★CODE 1★] # text에 한국어 형태소 분석 적용
        tokens_a = [★CODE 2★] # otk_tokenized_text에 BERT 토크나이즈 적용

        ## 주어진 최대 시퀀스 길이보다 긴 문장은 끝부분을 잘라냄
        ## [CLS], [SEP] 토큰이 반드시 포함되므로 max_seq_length - 2보다 긴 부분 자르기
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

        ## STEP 2. BERT 인풋 형식 맞추기 : [CLS] 문장 [SEP] 형태 만들기
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]

        ## STEP 3. segment_ids 만들기 
        ## -> 단일 문장이므로 모두 '0' 인덱스를 부여함
        segment_ids = [★CODE 3★]

        ## STEP 4. 토큰을 input_ids로 변환
        ## hint: tokenizer.convert_tokens_to_ids 사용
        input_ids = [★CODE 4★]

        ## STEP 5. 인풋 마스크 만들기 
        ## -> 실제 토큰이 있는 부분을 1로 채움
        input_mask = [★CODE 5★]

        ## max_seq_length보다 모자른 부분을 padding으로 채움 
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if ex_index < 3:
            print("*** Example ***")
            print("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))

            print("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            print("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            print("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            print("label: %d" % (label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features


#### Step 3. BERT 인풋 만들기 - 데이터에 함수 적용
- 이제 정의한 convert_examples_to_features 함수를 이용해 우리의 train 데이터를 BERT인풋으로 만듭니다.
- 먼저 176개의 라벨을 0~175의 정수 인덱스로 매핑하는 label_map을 생성합니다.
- 이후 train 데이터에 함수를 적용해 train_features라는 변수에 저장하겠습니다.

<font color = "red">[MISSION] 전처리 결과 input_ids, input_mask, segment_id가 잘 생성되었는지 확인하세요

In [ ]:
# label mapping 사전 만들기
label_map = {}
label_id = 0
for dat in train:
  if dat[2] not in label_map:
    label_map[dat[2]] = label_id
    label_id += 1

## 최대 문장 길이를 128으로 설정하고 feature 만들기

MAX_SEQ_LENGTH = 128
print("*** Convert Train Examples ***")
train_features = convert_examples_to_features(train, MAX_SEQ_LENGTH, tokenizer, label_map)

*** Convert Train Examples ***
*** Example ***
tokens: [CLS] 뭔 ##가 하 ##루 종 ##일 이 ##렇게 들 ##뜬 기 ##분 이다 보 니 ##까 잠 ##도 잘 안 와 . [SEP]
input_ids: 101 9304 11287 9952 35866 9684 18392 9638 82838 9117 118847 8932 37712 30919 9356 9049 118671 9655 12092 9654 9521 9590 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
print(label_map)

{'증상/불면': 0, '감정/분노': 1, '증상/기절': 2, '감정/불안감': 3, '증상/두근거림': 4, '배경/건강문제': 5, '배경/취업': 6, '감정/우울감': 7, '배경/대인관계': 8, '배경/어린시절': 9, '감정/자살충동': 10, '감정/외로움': 11, '증상/자살시도': 12, '감정/걱정': 13, '배경/학업': 14, '증상/무기력': 15, '증상/식욕저하': 16, '일반대화': 17, '배경/결혼': 18, '상태/증상지속': 19, '배경/학교': 20, '감정/짜증': 21, '배경/부모': 22, '증상/두통': 23, '증상/피해망상': 24, '감정/자존감저하': 25, '배경/성격': 26, '증상/기억상실': 27, '배경/사업': 28, '감정/감정조절이상': 29, '배경/전연인': 30, '감정/불편감': 31, '배경/경제적문제': 32, '치료이력/병원내원': 33, '배경/친구': 34, '배경/가족': 35, '배경/여자친구': 36, '치료이력/검사': 37, '증상/어지러움': 38, '감정/부정적사고': 39, '감정/눈물': 40, '배경/시댁': 41, '증상/반복행동': 42, '감정/좌절': 43, '배경/자녀': 44, '감정/무력감': 45, '배경/사고': 46, '배경/직장': 47, '증상/기억력저하': 48, '증상/은둔': 49, '증상/환청': 50, '증상/이명': 51, '부가설명': 52, '감정/심란': 53, '감정/답답': 54, '감정/힘듦': 55, '감정/공허감': 56, '배경/남편': 57, '증상/반복사고': 58, '감정/무서움': 59, '감정/두려움': 60, '증상/피로': 61, '감정/속상함': 62, '감정/통제력상실': 63, '배경/생활': 64, '배경/종교': 65, '감정/자괴감': 66, '증상/환각': 67, '배경/음주': 68, '배경/대학': 69, '자가치료/심리조절': 70, '감정/당황': 71, '감정/즐거

#### Step 4. 전처리 결과 확인 & 드라이브에 저장
- BERT 토크나이징 결과, [UNK]가 얼마나 생성되었는지 확인해보겠습니다.
- 현재 데이터의 경우, 학습 데이터 중 1.34%가 [UNK] 토큰으로 떨어진 것을 확인할 수 있습니다.
- 구글의 모델은 102국어에 대해 학습된 모델으로, 한국어만으로 학습한 모델에 비해 UNK 비율이 높은 편입니다.

In [ ]:
tot_tokens = 0
num_oov = 0
for feat in train_features:
  num_oov += sum([s == 100 for s in feat.input_ids])
  tot_tokens += sum([s != 0 for s in feat.input_ids])

print("학습 데이터 중 {}개 ({:.2f}%) 토큰은 [UNK] 처리됨".format(num_oov, 100*num_oov/tot_tokens))

학습 데이터 중 1351개 (1.34%) 토큰은 [UNK] 처리됨


In [ ]:
import pickle
with open("/content/gdrive/My Drive/NLP/BERT_CLS_train_features.pkl", "wb") as f:
  pickle.dump(train_features, f)